<a href="https://colab.research.google.com/github/aknip/llm-benchmark/blob/main/llm_simple_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

see https://www.philschmid.de/evaluate-llm

In [ ]:
!pip install pandas openpyxl

In [ ]:
import pandas as pd

def read_and_process_excel(file_path):
    # Read the Excel file into a Pandas DataFrame
    df = pd.read_excel(file_path)

    # Loop through the DataFrame and print names and phone numbers
    for index, row in df.iterrows():
        name = row['name']  # Assuming the column for names is labeled 'name'
        phone_number = row['phone number']  # Assuming the column for phone numbers is labeled 'phone number'
        print(f"{name}, {phone_number}")

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'your_file.xlsx'
read_and_process_excel(file_path)